# MLFlow runs

In [2]:
import mlflow
from mlflow.tracking import MlflowClient
import itertools
import matplotlib.pyplot as plt
from mlflow.entities import ViewType

mlflow.tracking.set_tracking_uri('../mlruns')

## Functions

In [3]:
def get_runs(experiment_name, filter_string=''):
    client = MlflowClient()
    exp_ids = [client.get_experiment_by_name(experiment_name).experiment_id]
    runs = mlflow.search_runs(experiment_ids=exp_ids, filter_string=filter_string)
    return runs


def get_best_run(runs, best_metrics):
    if runs is None or runs.shape[0] == 0:
        return None
    for metric in best_metrics:
        runs = runs.loc[runs[f'metrics.{metric}'] >= runs[f'metrics.{metric}'].max()]
    # pick first in the list
    run = runs.loc[runs.index[0]]
    return run

In [4]:
def runs_to_latex(runs, threshold_method, caption, label, grouped=False):
    cols = ['tags.mlflow.runName', 'tags.model', 'tags.method', 'metrics.test_auc', 'metrics.test_tax_yield', 'metrics.test_tax_score', 'metrics.test_effort', 'metrics.test_f1-score_1', 'metrics.test_recall_1', 'metrics.test_precision_1', 'metrics.threshold']
    cols_renamed = [ 'name', 'model', 'method', 'auc', 'yield', 'value', 'effort', 'f1-score', 'recall', 'precision', 'threshold' ]
    to_replace = {
        'model': {'tree': 'DT', 'forest': 'RF', 'logit': 'LR'},
        'method': {'imbalanced': 'IM', 'oversample': 'RO', 'weighted': 'BC'}}
    df = runs.loc[:, :]
    df = df.loc[df['tags.threshold_method']==threshold_method]
    df = df.loc[df['tags.grouped']==str(grouped)]
    df = df.sort_index(ascending=False)
    df = df.loc[:, cols]
    df = df.rename(columns=dict(zip(cols, cols_renamed)))
    df = df.replace(to_replace=to_replace)
    df.loc[:, 'name'] = df.model.str.cat(df.method, sep='-')
    df = df.drop(columns=['model', 'method']).set_index(['name'])
    if threshold_method=='default':
        df = df.drop(columns=['threshold'])
    else:
        df = df.drop(columns=['auc'])
    cols_max = df.columns.drop('effort').tolist()
    s = df.style.highlight_max(subset=cols_max, axis=0, props='bfseries: ;')
    s = s.highlight_min(subset='effort', axis=0, props='bfseries: ;')
    latex_table = s.to_latex(
        column_format="lrrrrrrr", position="ht", position_float="centering",
        hrules=True, label=label, caption=caption,
        multirow_align="t", multicol_align="r")
    return latex_table

## Reports

In [62]:
filter_string = 'tags.dataset = "SYNTHETIC" and tags.dataset_spec = "d4b" and status = "FINISHED" and tags.architecture_type = "cam"'
runs = get_runs('wsol', filter_string=filter_string)

In [63]:
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.test_accuracy,metrics.val_loss,metrics.val_MaxBoxAccV3,metrics.val_MaxBoxAccV3_IOU_30,...,tags.model,tags.wsol_method,tags.mlflow.project.backend,tags.optimizer,tags.pretrained,tags.architecture,tags.mlflow.log-model.history,tags.mlflow.source.git.repoURL,tags.mlflow.gitRepoURL,tags.mlflow.source.git.commit
0,e79d5e097d044114bff1f05ae5e37d4b,444281214894994099,FINISHED,file:///Users/goemaereg/github.com/thesis/mlru...,2023-02-04 20:55:28.487000+00:00,2023-02-04 21:03:49.365000+00:00,0.95,0.053912,0.167917,0.30125,...,VggCam,minmaxcam,local,SGD,True,vgg16,"[{""run_id"": ""e79d5e097d044114bff1f05ae5e37d4b""...",None,None,None
1,316d72602308482ca7c60e49bad2e84d,444281214894994099,FINISHED,file:///Users/goemaereg/github.com/thesis/mlru...,2023-02-02 13:36:48.472000+00:00,2023-02-02 13:51:33.719000+00:00,0.99,0.002068,0.266667,0.47625,...,VggCam,cam,local,SGD,True,vgg16,"[{""run_id"": ""316d72602308482ca7c60e49bad2e84d""...",None,None,None
2,e06bf522e10944f79b6f2a596d132b2f,444281214894994099,FINISHED,file:///Users/goemaereg/github.com/thesis/mlru...,2023-01-19 22:22:28.980000+00:00,2023-01-19 22:25:10.701000+00:00,0.99,0.002068,0.160833,0.30750,...,VggCam,cam,local,SGD,True,vgg16,"[{""run_id"": ""e06bf522e10944f79b6f2a596d132b2f""...",git@github.com:goemaereg/thesis.git,git@github.com:goemaereg/thesis.git,0a34ae3aa7ee557ac0f16b28bae58b5f453b34f0
3,5106e27d6d7648048e60261474f7992b,444281214894994099,FINISHED,file:///Users/goemaereg/github.com/thesis/mlru...,2023-01-19 20:59:25.608000+00:00,2023-01-19 21:01:07.397000+00:00,0.99,0.002068,0.160833,0.30750,...,VggCam,cam,local,SGD,True,vgg16,"[{""run_id"": ""5106e27d6d7648048e60261474f7992b""...",git@github.com:goemaereg/thesis.git,git@github.com:goemaereg/thesis.git,0a34ae3aa7ee557ac0f16b28bae58b5f453b34f0


In [64]:
cols = ['params.epochs','tags.wsol_method', 'tags.cam_method', 'metrics.test_MaxBoxAccV3', 'metrics.test_PxAP']

In [65]:
runs.loc[:, cols].to_dict()


{'params.epochs': {0: '10', 1: '10', 2: '10', 3: '10'},
 'tags.wsol_method': {0: 'minmaxcam', 1: 'cam', 2: 'cam', 3: 'cam'},
 'tags.cam_method': {0: 'cam', 1: 'scorecam', 2: 'gradcam', 3: 'cam'},
 'metrics.test_MaxBoxAccV3': {0: 0.16749999999999998,
  1: 0.26499999999999996,
  2: 0.15708333333333332,
  3: 0.15708333333333332},
 'metrics.test_PxAP': {0: 0.3128760840363067,
  1: 0.39582912166223805,
  2: 0.296918513557799,
  3: 0.29691838737908316}}